In [1]:
## packages
import pandas as pd
import numpy as np
import tarfile
import json
import random

## unzip tar
# import tarfile
# tar = tarfile.open('yelp_dataset.tarf]=', 'r:gz')
# tar.extractall()
# tar.close()

# set random seed for consistent results
random.seed(10)

In [2]:
# reviews = pd.read_json('yelp_academic_dataset_review.json', lines = True)
# users = pd.read_json('yelp_academic_dataset_user.json', lines = True)
# businesses = pd.read_json('yelp_academic_dataset_business.json', lines = True)
philly_bus = pd.read_feather('../FilteredData/business_philly.feather')
philly_reviews = pd.read_feather('../FilteredData/review_philly.feather')
philly_users = pd.read_feather('../FilteredData/user_philly.feather')

In [3]:
# business_cat = businesses.categories.unique
# df = businesses[businesses['categories'].str.contains('Restaurant')]
# philly_bus
philly_users
# philly_users["average_stars_given"].dtypes

,index,_id,user_id,name,review_count,yelping_since,useful,funny,cool,elite,...,compliment_writer,compliment_photos,gender_score,philly_reviews,philly_share_of_reviews,bucketed_philly_share_of_reviews,bucketed_gender_scores,average_stars_given,bucketed_average_stars_given,bucketed_philly_reviews
0,4,631ea1f7abab926ea88770d7,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...",...,1521,1946,0.995754,13,0.003000,less_than_or_equal_to_25_percent,more_than_80_percent,4.0,more_than_3_up_to_4,more_than_5_up_to_25
1,6,631ea1f7abab926ea88770d9,AUi8MPWJ0mLkMfwbui27lg,John,109,2010-01-07 18:32:04,154,20,23,,...,0,0,0.995785,2,0.018349,less_than_or_equal_to_25_percent,more_than_80_percent,2.0,more_than_1_up_to_2,more_than_1_up_to_5
2,12,631ea1f7abab926ea88770df,1McG5Rn_UDkmlkZOrsdptg,Teresa,7,2009-05-26 16:11:11,18,3,13,,...,0,0,0.002806,5,0.714286,more_than_50_up_to_75_percent,less_than_or_equal_to_20_percent,4.0,more_than_3_up_to_4,more_than_1_up_to_5
3,21,631ea1f7abab926ea88770e8,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1221,2005-03-14 20:26:35,14953,9940,11211,"2006,2007,2008,2009,2010,2011,2012,2013,2014",...,815,323,0.003043,4,0.003276,less_than_or_equal_to_25_percent,less_than_or_equal_to_20_percent,4.0,more_than_3_up_to_4,more_than_1_up_to_5
4,22,631ea1f7abab926ea88770e9,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,...,239,180,0.986824,3,0.005128,less_than_or_equal_to_25_percent,more_than_80_percent,5.0,more_than_4,more_than_1_up_to_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245406,1987805,631ea2f2abab926ea8a5c5b0,Tr2yeddopDzMhSb7jbdNeA,Heather,1,2017-07-25 19:31:30,0,0,0,,...,0,0,0.002816,1,1.000000,more_than_75_percent,less_than_or_equal_to_20_percent,5.0,more_than_4,just_1
245407,1987831,631ea2f2abab926ea8a5c5ca,XTWARBzLbiJKQ1JeJB9f3g,Sakina,6,2017-11-24 03:17:11,1,0,0,,...,0,0,0.000000,1,0.166667,less_than_or_equal_to_25_percent,less_than_or_equal_to_20_percent,4.0,more_than_3_up_to_4,just_1
245408,1987832,631ea2f2abab926ea8a5c5cb,iZ0puydkFQlaSQAXbGtv1g,Dr. Sylvia,1,2014-07-08 19:54:19,2,0,0,,...,0,0,0.500000,1,1.000000,more_than_75_percent,more_than_40_up_to_60_percent,5.0,more_than_4,just_1
245409,1987859,631ea2f2abab926ea8a5c5e6,K-BNvWUCWEGUuBirCU4mmg,Jodi,2,2015-03-23 17:29:22,0,0,0,,...,0,0,0.011541,1,0.500000,more_than_25_up_to_50_percent,less_than_or_equal_to_20_percent,5.0,more_than_4,just_1


In [4]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k,auc_score,reciprocal_rank
from lightfm.data import Dataset
from lightfm import LightFM, cross_validation

C:\Users\carlee\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [5]:
# df = philly_reviews.groupby('user_id')['stars'].mean()
# users = pd.merge(philly_users, df, on=['user_id'], how='left')

In [6]:
# bins = [0, 0.9999999, 1.9999999, 2.9999999, 3.9999999, 4.9999999, 5]
# labels = ["0","1", "2", "3","4", "5"]
# users["star_bin"] = pd.cut(users['stars'], bins=bins, labels=labels)
# # users.loc[(users['stars'] > 4.9999999) & (users['stars'] < 5)]

In [7]:
# # list(filter(lambda x: '27281' in x, uf))
user_ft = philly_users[["philly_share_of_reviews", "gender_score", "philly_reviews", "bucketed_average_stars_given"]]

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

x = user_ft
x = x.replace([np.inf, -np.inf], np.nan)
x = x.fillna(0)
y = x.pop("bucketed_average_stars_given")

In [9]:
# train_data, test_data, train_label, test_label = train_test_split(x, y, test_size=0.25, stratify=y)


# rf_model = RandomForestClassifier()
# rf_model.fit(train_data, train_label)
# feature_importance = pd.DataFrame({"features": train_data.columns,
#                                    "importance": rf_model.feature_importances_})
# feature_importance.sort_values("importance", ascending=False).head(15)

In [10]:
#unique user features
user_f = []
user_col = ['bucketed_philly_share_of_reviews']*len(philly_users['bucketed_philly_share_of_reviews'].unique()) + ['bucketed_gender_scores']*len(philly_users['bucketed_gender_scores'].unique()) + ['bucketed_average_stars_given']*len(philly_users['bucketed_average_stars_given'].unique())  + ['bucketed_philly_reviews']*len(philly_users['bucketed_philly_reviews'].unique()) 
user_unique_list = list(philly_users['bucketed_philly_share_of_reviews'].unique()) + list(philly_users['bucketed_gender_scores'].unique()) + list(philly_users['bucketed_average_stars_given'].unique()) + list(philly_users['bucketed_philly_reviews'].unique())
# user_col = ['bucketed_philly_share_of_reviews']*len(philly_users['bucketed_philly_share_of_reviews'].unique()) + ['bucketed_gender_scores']*len(philly_users['bucketed_gender_scores'].unique()) + ['bucketed_philly_reviews']*len(philly_users['bucketed_philly_reviews'].unique()) 
# user_unique_list = list(philly_users['bucketed_philly_share_of_reviews'].unique()) + list(philly_users['bucketed_gender_scores'].unique()) + list(philly_users['bucketed_philly_reviews'].unique())


for x,y in zip(user_col, user_unique_list):
    res = str(x)+ ":" +str(y)
    user_f.append(res)
#     print(res)

In [11]:
#unique item features
item_f = []
item_col = ['bucketed_average_stars_received']*len(philly_bus['bucketed_average_stars_received'].unique()) + ['postal_code']*len(philly_bus['postal_code'].unique()) + ['bucketed_sentiment_scores_received']*len(philly_bus['bucketed_sentiment_scores_received'].unique()) + ['bucketed_review_count']*len(philly_bus['bucketed_review_count'].unique())
item_unique_list = list(philly_bus['bucketed_average_stars_received'].unique()) + list(philly_bus['postal_code'].unique()) + list(philly_bus['bucketed_sentiment_scores_received'].unique()) + list(philly_bus['bucketed_review_count'].unique()) 
 
for x,y in zip(item_col, item_unique_list):
    res = str(x)+ ":" +str(y)
    item_f.append(res)
#     print(res)

In [12]:
dataset1 = Dataset()
dataset1.fit(
        philly_reviews['user_id'].unique(), # all the users
        philly_reviews['business_id'].unique(), # all the items
        user_features = user_f, # additional user features
        item_features = item_f #additional item features
)

In [13]:
reviews_only = philly_reviews[["user_id", "business_id", "stars"]]
(interactions, weights) = dataset1.build_interactions([(x[0], x[1], x[2]) for x in reviews_only.values])

In [14]:
def user_feature_colon_value(my_list):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,0,'del'],
    resultant output = ['f1:1', 'f2:1', 'f3:0', 'loc:del']
   
    """
    result = []
    ll = ['bucketed_philly_share_of_reviews:', 'bucketed_gender_scores:', 'bucketed_average_stars_given:','bucketed_philly_reviews:']
    # ll = ['review_count:', 'useful:', 'funny:', 'cool:']
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result
# Using the helper function to generate user features in proper format for ALL users
user_ad_subset = philly_users[['bucketed_philly_share_of_reviews', 'bucketed_gender_scores', 'bucketed_average_stars_given','bucketed_philly_reviews']] 
# ad_subset = philly_users[["review_count",'useful', 'funny', 'cool']] 
user_ad_list = [list(x) for x in user_ad_subset.values]
user_feature_list = []
for item in user_ad_list:
    user_feature_list.append(user_feature_colon_value(item))
# print(f'Final output: {feature_list}')
user_tuple = list(zip(philly_users.user_id, user_feature_list))
user_features = dataset1.build_user_features(user_tuple, normalize= False)


In [15]:
eye = np.eye(user_features.shape[0], user_features.shape[0]).tocsr()
user_features_concat = np.hstack((eye, user_features))
user_features = user_features_concat.tocsr().astype(np.float32) 

MemoryError: Unable to allocate 449. GiB for an array with shape (245411, 245411) and data type float64

In [ ]:
def item_feature_colon_value(my_list):
    result = []
    ll = ['bucketed_average_stars_received:', 'postal_code:', 'bucketed_sentiment_scores_received:','bucketed_review_count:']
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result
# Using the helper function to generate user features in proper format for ALL users
item_ad_subset = philly_bus[['bucketed_average_stars_received', 'postal_code', 'bucketed_sentiment_scores_received','bucketed_review_count']] 
item_ad_list = [list(x) for x in item_ad_subset.values]
item_feature_list = []
for item in item_ad_list:
    item_feature_list.append(item_feature_colon_value(item))
item_tuple = list(zip(philly_bus.business_id, item_feature_list))
item_features = dataset1.build_item_features(item_tuple, normalize= False)


In [ ]:
(interactions, weights) = dataset1.build_interactions([(x[0], x[1], x[2]) for x in reviews_only.values])

Train/Test Split

In [ ]:
train, test = cross_validation.random_train_test_split(interactions, test_percentage=0.25)

Model with no features

In [ ]:
model = LightFM()
model.fit(train,
      epochs=30,
      num_threads=4)

In [ ]:
from lightfm.evaluation import auc_score
test_auc = auc_score(model,
                      test,
                     ).mean()
print('Test set AUC: %s' % test_auc)

Model with user features only

In [ ]:
model2 = LightFM() #loss='warp'
model2.fit(train,
      user_features= user_features,
      # item_features = item_features
      epochs=30,
      num_threads=4)

In [ ]:
test_auc2 = auc_score(model2,
                      test,
                      user_features=user_features
                     ).mean()
print('Hybrid test set (user features only) AUC: %s' % test_auc2)
#0.77177984

Model with item features only

In [ ]:
model3 = LightFM() #loss='warp'
model3.fit(train,
      # user_features= user_features,
      item_features = item_features,
      epochs=30,
      num_threads=4)

In [ ]:
test_auc3 = auc_score(model3,
                      test,
                    #   user_features=user_features,
                      item_features=item_features
                     ).mean()
print('Hybrid test set (item features only) AUC: %s' % test_auc3)

Model with user and item feature

In [ ]:
model4 = LightFM() #loss='warp'
model4.fit(train,
      user_features= user_features,
      item_features = item_features,
      epochs=30,
      num_threads=4)

In [ ]:
test_auc4 = auc_score(model4,
                      test,
                      user_features=user_features,
                      item_features=item_features
                     ).mean()
print('Hybrid test set (user & item features) AUC: %s' % test_auc4)